In [1]:
# First we will import the necessary Library 
import os
import pandas as pd
import numpy as np
import datetime as dt
from itertools import cycle
import plotly.express as px

In [2]:
#Load the dataset
data = pd.read_excel("Data.xlsx")

pd.set_option('display.max_rows', None)

In [3]:
data.head()

,Date,"Domestic Market (Contract) Blow Molding, Low",Spot/Export Blow Molding,"Spot, Domestic",WTISPLC,MCOILBRENTEU,GASREGM,IMPCH,EXPCH,PRUBBUSDM,...,Russia_import,South_Africa_import,Turkey_import,Brazil_import,France_import,Germeny_import,United Kingdome_import,China_import,Japan_import,South_korea_import
0,2000-01-01,41.0,NaN,NaN,27.18,25.51,1.289,6902.1,863.1,29.207387,...,NaN,NaN,6833222.0,2649293.0,35239920.0,NaN,NaN,NaN,NaN,NaN
1,2000-02-01,41.0,NaN,NaN,29.35,27.78,1.377,6584.4,972.7,33.391099,...,NaN,NaN,5647987.0,2787667.0,36716752.0,NaN,NaN,NaN,NaN,NaN
2,2000-03-01,45.0,NaN,NaN,29.89,27.49,1.516,6424.1,1330.5,30.941913,...,NaN,NaN,6721125.0,4692701.0,45144561.0,NaN,NaN,NaN,NaN,NaN
3,2000-04-01,47.0,NaN,NaN,25.74,22.76,1.465,7070.5,1227.5,31.930148,...,NaN,NaN,7169182.0,2565068.0,32240349.0,NaN,NaN,NaN,NaN,NaN
4,2000-05-01,47.0,NaN,NaN,28.78,27.74,1.487,7850.2,1526.3,31.201702,...,NaN,NaN,10466333.0,4671325.0,29351633.0,NaN,NaN,NaN,NaN,NaN


In [4]:
data['Domestic Market (Contract) Blow Molding, Low'].isnull().sum()

0

In [5]:
df=data.loc[:,'Domestic Market (Contract) Blow Molding, Low']

In [6]:
from sklearn.preprocessing import MinMaxScaler

#normalizing using MinMax Scaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df).reshape(-1,1))
print(df1.shape)

(276, 1)


In [7]:
training_size=int(len(df1)*0.80)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (220, 1)
test_data:  (56, 1)


In [8]:
from tensorflow.keras.models import load_model
model1 = load_model('model.h5')

In [9]:
time_step = 5

# Forecasting for next 30 days

In [10]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
#print(temp_input)
temp_input=temp_input[0].tolist()
#print(temp_input)

from numpy import array

lst_output=[]
n_steps=time_step
#print(time_step)
i=0
pred_days = 30
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("\nDay {} input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model1.predict(x_input, verbose=0)
        #print("Day {} output {}".format(i,yhat[0]))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(len(temp_input))
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        #print(x_input)
        yhat = model1.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
    
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  30


# Plotting the forecasted output

In [11]:
pre_days = 30

In [12]:
temp_mat = np.empty((pre_days+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:pre_days] = scaler.inverse_transform(df1[len(df1)-pre_days:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[pre_days:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 30 days with Predicted next 15 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value']],
              labels={'value': 'Domestic Market (Contract) Blow Molding, Low','index': 'Timestamp'})
fig.update_layout(title_text='Comparing last 30 days vs next 15 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()